In [7]:
import torch, torchtext, numpy as np
import pandas as pd, csv
from torch import nn, optim
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pdb
from keras.models import Sequential
from tensorflow import keras
from scipy import signal
import os

In [8]:
# assign directory
directory = "drive/MyDrive/MINT_SHAPE_DATA"
 
# iterate over files in
# that directory
dataset=[]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        ds = pd.read_csv(f)
        i=ds.loc[ds[' EXG Channel 0'] == 0].index
        if(len(i)==1):
          ds=ds.tail(-(i[0]+1))
        dataset.append(ds)

In [9]:
dataset[6]

,Sample Index,EXG Channel 0,EXG Channel 1,EXG Channel 2,EXG Channel 3,EXG Channel 4,EXG Channel 5,EXG Channel 6,EXG Channel 7,Accel Channel 0,...,Other.3,Other.4,Other.5,Other.6,Analog Channel 0,Analog Channel 1,Analog Channel 2,Timestamp,Other.7,Timestamp (Formatted)
123,1.0,-134537.630920,-164114.040329,12766.400011,-120367.004915,74866.564556,-75084.337602,48114.007487,-114896.884846,-0.034,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666406e+09,0.0,2022-10-21 19:30:13.506
124,2.0,-134563.134261,-164128.144279,12772.971424,-120412.960102,74834.176878,-75108.544541,48094.427358,-114924.824527,-0.034,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666406e+09,0.0,2022-10-21 19:30:13.506
125,3.0,-134573.125490,-164132.458166,12785.622512,-120434.753053,74811.221637,-75156.220812,48073.461422,-114944.985800,-0.034,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666406e+09,0.0,2022-10-21 19:30:13.507
126,4.0,-134564.028330,-164122.556343,12830.683628,-120437.234096,74819.312968,-75163.999219,48089.420568,-114917.582562,-0.034,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666406e+09,0.0,2022-10-21 19:30:13.507
127,5.0,-134567.694016,-164123.361006,12861.551387,-120463.050361,74810.998119,-75202.623034,48087.833594,-114915.302684,-0.034,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666406e+09,0.0,2022-10-21 19:30:13.507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7814,12.0,-136063.383348,-164002.147496,9005.137861,-120830.200115,74507.394374,-91886.032151,46260.891409,-115026.256743,0.038,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666406e+09,0.0,2022-10-21 19:30:43.727
7815,13.0,-136064.120956,-163999.487638,9011.172832,-120811.894037,74511.931778,-91876.555011,46266.345235,-115023.619237,0.038,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666406e+09,0.0,2022-10-21 19:30:43.727
7816,14.0,-136066.937276,-164003.533304,9008.535327,-120814.106859,74506.992043,-91871.905848,46266.747566,-115016.578438,0.038,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666406e+09,0.0,2022-10-21 19:30:43.727
7817,15.0,-136063.271590,-164006.126107,9004.310847,-120831.854145,74496.553778,-91888.870822,46255.102307,-115021.249952,0.038,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666406e+09,0.0,2022-10-21 19:30:43.727


In [10]:
for data in dataset:
  data.drop(data.iloc[:, 9:25], axis=1, inplace=True)
dataset[6].shape

(7696, 9)

In [19]:
i =0;
for data in dataset:
  if data.shape[0] < 6144:
    i=i+1
i

0

In [20]:
len(dataset)

32

In [21]:
data_train=dataset[0:20]
data_test=dataset[20:32]

In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler =MinMaxScaler()
i=0;
for data in dataset[0:20]:
  data_train[i] = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
  i=i+1
i=0
for data in dataset[20:32]:
  data_test[i] = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
  i=i+1

In [23]:
import tqdm #0 is big
n=256*6
feature_names=list(data_train[0].drop('Sample Index',axis=1).columns)
X=[]
Y=[]
indexes=[]
for data in data_train:
  norm_data_x =data[feature_names]
  for i in tqdm.tqdm_notebook(range(4)):
    X.append(norm_data_x.iloc[i*n:i*n+n].  values)
    Y.append(i%2)
    indexes.append(data.index[i+n-1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [24]:
print(X[39].shape)
print(Y[39])

(1536, 8)
1


In [42]:
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import adam_v2
n_steps=256*6
n_features=8
model=Sequential()
model.add(LSTM(100, activation='relu', return_sequences=False, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(Dense(1))
opt = adam_v2.Adam(lr=0.0005, decay=1e-6)

model.compile(optimizer="adam", loss='mse', metrics=['mape'])

In [43]:
history=model.fit(np.asarray(X),np.asarray(Y),batch_size=20,epochs=13)

Epoch 1/13
4/4 [==============================] - 4s 753ms/step - loss: 0.3506 - mape: 161233440.0000
Epoch 2/13
4/4 [==============================] - 3s 744ms/step - loss: 0.2943 - mape: 241854112.0000
Epoch 3/13
4/4 [==============================] - 3s 737ms/step - loss: 0.2884 - mape: 244850464.0000
Epoch 4/13
4/4 [==============================] - 3s 733ms/step - loss: 0.2642 - mape: 210384720.0000
Epoch 5/13
4/4 [==============================] - 3s 735ms/step - loss: 0.2633 - mape: 191818832.0000
Epoch 6/13
4/4 [==============================] - 3s 736ms/step - loss: 0.2415 - mape: 187293024.0000
Epoch 7/13
4/4 [==============================] - 3s 734ms/step - loss: 0.2104 - mape: 151840480.0000
Epoch 8/13
4/4 [==============================] - 3s 723ms/step - loss: 0.1789 - mape: 146469488.0000
Epoch 9/13
4/4 [==============================] - 3s 733ms/step - loss: 0.1765 - mape: 141978160.0000
Epoch 10/13
4/4 [==============================] - 3s 737ms/step - loss: 0.1756 - 

In [44]:
n=256*6
feature_names=list(data_test[0].drop('Sample Index',axis=1).columns)
X_test=[]
Y_test=[]
indexes=[]
for data in data_test:
  norm_data_x =data[feature_names]
  for i in tqdm.tqdm_notebook(range(4)):
    X_test.append(norm_data_x.iloc[i*n:i*n+n].  values)
    Y_test.append(i%2)
    indexes.append(data.index[i+n-1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [45]:
predictions = model.predict(np.asarray(X_test))
predictions = pd.DataFrame(predictions).rename(columns={0: 'prediction'})
actual=pd.DataFrame(Y_test).rename(columns={0: 'actual'})
final=pd.concat([predictions,actual],axis=1)
final['mae']=abs(final['prediction']-final['actual'])
final

2/2 [==============================] - 1s 176ms/step


,prediction,actual,mae
0,-0.086643,0,0.086643
1,0.739246,1,0.260754
2,0.488248,0,0.488248
3,0.756632,1,0.243368
4,1.107252,0,1.107252
5,1.428610,1,0.428610
6,1.312568,0,1.312568
7,1.608386,1,0.608386
8,-0.092400,0,0.092400
9,0.622664,1,0.377336


In [46]:
import pickle
filename = 'finalized_shape_model.sav'
pickle.dump(model, open(filename, 'wb'))